In [1]:
import torch

from llmtosql.model import WikiSQLModel
from llmtosql.trainer import Trainer
from llmtosql.dataloader import WikiSQLDataset
from llmtosql.utils.utils import plot_history, load_model, load_history
from matplotlib import pyplot as plt
import sys
import os
import json
import numpy as np
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.insert(0, '..')

In [2]:
model = WikiSQLModel(base_model_type='bert-base-uncased', heads=(False, False, True))
# try:
#     model = load_model(model, 'model_output/model.pth')
# except Exception as e:
#     print(f'No model loaded - {e}')

2023-05-08 19:43:40 [info     ] Using cross attention mechanism
2023-05-08 19:43:40 [info     ] 1 heads model -- ['CONDS']


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
train_set =  WikiSQLDataset(type='dev', model=model)

2023-05-08 19:43:43 [info     ] Tokenizing dataset.


100%|██████████| 8421/8421 [00:06<00:00, 1229.86it/s]


In [4]:
train_set.maxcondsLength

4

In [5]:
# val_set = WikiSQLDataset(type='dev')

In [6]:
len(train_set.data)

8421

In [7]:
# datasets = (train_set, val_set)

In [8]:
config = {
    'seed': 32,
    'scheduler': None,
    'optimizer': 'adam',
    'momentum': 0.9,
    'weight_decay': 0.0,
    'lr': 0.0005,
    'criterion': 'custom',
    'metric': 'accuracy',
    'model_dir': 'model_output',
    'backend': 'smddp'
}

In [9]:
trainer = Trainer(model, datasets=train_set,
                  epochs=10, batch_size=2,
                  is_parallel=False,
                  custom_model=True,
                  **config)

2023-05-08 19:43:49 [info     ] Config inputs.                 config={'seed': 32, 'scheduler': None, 'optimizer': 'adam', 'momentum': 0.9, 'weight_decay': 0.0, 'lr': 0.0005, 'criterion': 'custom', 'metric': 'accuracy', 'model_dir': 'model_output', 'backend': 'smddp'}
2023-05-08 19:43:49 [info     ] Loading the model.
2023-05-08 19:43:49 [info     ] Loading training and validation set.
2023-05-08 19:43:49 [info     ] Preparing the data.
2023-05-08 19:43:49 [debug    ] Processes 8421/8421 (100%) of train data
2023-05-08 19:43:49 [info     ] Training on device: cpu.
2023-05-08 19:43:49 [info     ] Using CUSTOM loss
2023-05-08 19:43:49 [info     ] Using ADAM optimizer


In [10]:
trainer.fit()

2023-05-08 19:43:49 [info     ] Start training..
2023-05-08 19:43:49 [info     ] ------------------------------ EPOCH 1 / 10 ------------------------------


  0%|          | 0/4211 [00:03<?, ?batch/s]


TypeError: expected Tensor as element 0 in argument 0, but got list